In [1]:
!pip install 'shimmy>=2.0'
!pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [11]:
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from scipy.sparse import issparse, csr_matrix
from scipy.sparse.linalg import eigsh, splu
from scipy.sparse import identity
from scipy.sparse import random as sparse_random

# Hybrid Shift-and-Invert Power Method + RL
class ShiftInvertPowerRL:
    def __init__(self, A, sigma=0.0, shift_tol=1e-6):
        self.A = A
        self.sigma = sigma  # Shift value
        self.shift_tol = shift_tol
        self.dim = A.shape[0]
        self._precompute_shifted_matrix()

    def _precompute_shifted_matrix(self):
        """Precompute (A - σI)^-1 for sparse systems"""
        if issparse(self.A):
            I = identity(self.dim, format='csr')
            self.M = self.A - self.sigma * I
            self.M_solver = splu(self.M.tocsc())
        else:
            self.M = self.A - self.sigma * np.eye(self.dim)
            self.M_inv = np.linalg.inv(self.M)

    def solve_shifted_system(self, b):
        """Solve (A - σI)x = b"""
        if issparse(self.A):
            return self.M_solver.solve(b)
        else:
            return np.linalg.solve(self.M, b)

    def hybrid_power_iteration(self, num_iter=10):
        """Warm start using shift-and-invert power method"""
        x = np.random.randn(self.dim)
        x /= np.linalg.norm(x)

        for _ in range(num_iter):
            x = self.solve_shifted_system(x)
            x /= np.linalg.norm(x) + 1e-8
        return x

# RL Environment for Eigenvector Refinement
class ShiftInvertEnv(gym.Env):
    def __init__(self, size=1000, sigma=0.0, target_eigenvalue=None, density=0.01):
        super().__init__()
        self.size = size
        self.sigma = sigma
        self.target_eigenvalue = target_eigenvalue
        self.density = density
        self.dim = size
        self._generate_matrix()
        self.solver = ShiftInvertPowerRL(self.A, sigma)

        # Observation: current vector + A*v + A^2*v + residual stats
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf,
                                           shape=(self.dim * 3 + 3,), dtype=np.float32)

        # Action: vector adjustment directions
        self.action_space = spaces.Box(low=-1, high=1, shape=(self.dim,), dtype=np.float32)

        # Initialize with hybrid power method
        self.state = self.solver.hybrid_power_iteration(num_iter=20)
        self.residual_history = []
        self.A_norm = np.sqrt(np.sum(self.A.data**2)) if issparse(self.A) else np.linalg.norm(self.A, 'fro')

    def _generate_matrix(self):
        """Generate a random sparse symmetric matrix"""
        np.random.seed(None)  # Allow randomness for diversity
        self.A = sparse_random(self.size, self.size, density=self.density, format='csr')
        self.A = (self.A + self.A.T) * 0.5  # Ensure symmetry
        # Update target eigenvalue if not provided
        if self.target_eigenvalue is None:
            eigvals, _ = eigsh(self.A, k=1, sigma=self.sigma)
            self.target_eigenvalue = eigvals[0]

    def reset(self):
        self._generate_matrix()  # New matrix for each episode
        self.solver = ShiftInvertPowerRL(self.A, self.sigma)
        self.state = self.solver.hybrid_power_iteration(num_iter=20)
        self.residual_history = []
        self.A_norm = np.sqrt(np.sum(self.A.data**2)) if issparse(self.A) else np.linalg.norm(self.A, 'fro')
        return self._get_obs()

    def step(self, action):
        # RL-guided adjustment
        adjusted_vec = self.state + 0.1 * action
        adjusted_vec /= np.linalg.norm(adjusted_vec) + 1e-8

        # Power method step for stability
        refined_vec = self.solver.solve_shifted_system(adjusted_vec)
        refined_vec /= np.linalg.norm(refined_vec) + 1e-8

        # Calculate metrics
        residual = self.A @ refined_vec - self.target_eigenvalue * refined_vec
        residual_norm = np.linalg.norm(residual)
        self.residual_history.append(residual_norm)

        # Reward shaping, normalized by matrix norm
        reward = -residual_norm / (self.A_norm + 1e-8) - 0.1 * np.log(residual_norm + 1e-8)

        self.state = refined_vec
        done = residual_norm < self.solver.shift_tol

        return self._get_obs(), reward, done, {}

    def _get_obs(self):
        """Augment state with matrix-vector products and residual stats"""
        Av = self.A @ self.state if issparse(self.A) else self.A @ self.state
        A2v = self.A @ Av if issparse(self.A) else self.A @ Av
        residual_stats = [
            np.mean(self.residual_history[-10:]),
            np.min(self.residual_history[-10:]),
            np.std(self.residual_history[-10:])
        ] if self.residual_history else [0, 0, 0]
        return np.concatenate([self.state, Av, A2v, residual_stats])

# Training and Evaluation
def train_hybrid_eigen_solver(size=1000, sigma=0.0, total_timesteps=100000, density=0.01):
    # Initialize environment with dynamic matrix generation
    env = ShiftInvertEnv(size=size, sigma=sigma, density=density)

    # RL Configuration with deeper policy
    policy_kwargs = dict(net_arch=[512, 256, 128])
    model = PPO(
        "MlpPolicy",
        env,
        learning_rate=1e-4,
        gamma=0.99,
        batch_size=256,
        policy_kwargs=policy_kwargs,
        verbose=1
    )

    # Train on diverse matrices
    model.learn(total_timesteps=total_timesteps)

    # Save model for inference
    model.save("matrix_aware_eigen_solver")

    # Evaluate on a new matrix
    env._generate_matrix()
    obs = env.reset()
    best_vec = env.state
    best_residual = np.inf

    # Compute actual eigenvector for evaluation
    eigvals, eigvecs = eigsh(env.A, k=1, sigma=sigma)
    actual_eigenvector = eigvecs[:, 0]

    for _ in range(1000):
        action, _ = model.predict(obs)
        obs, _, _, _ = env.step(action)
        current_residual = np.linalg.norm(env.A @ env.state - env.target_eigenvalue * env.state)

        if current_residual < best_residual:
            best_residual = current_residual
            best_vec = env.state

    # Compute cosine distance
    v1 = best_vec / (np.linalg.norm(best_vec) + 1e-8)
    v2 = actual_eigenvector / (np.linalg.norm(actual_eigenvector) + 1e-8)
    cos_sim = np.dot(v1, v2)
    cos_sim = np.clip(cos_sim, -1.0, 1.0)  # Handle numerical errors
    cos_dist = 2 * np.arccos(np.abs(cos_sim)) / np.pi

    print(f"Final residual norm: {best_residual:.4e}")
    print(f"Eigenvector norm check: {np.linalg.norm(best_vec):.4f}")
    print(f"Cosine distance to actual eigenvector: {cos_dist:.6f}")

    return model, best_vec, best_residual, cos_dist

# Inference on a New Matrix
def infer_eigenvector(model, A, sigma=0.0):
    # Compute target eigenvalue
    eigvals, eigvecs = eigsh(A, k=1, sigma=sigma)
    target_eigenvalue = eigvals[0]
    actual_eigenvector = eigvecs[:, 0]

    # Initialize environment with provided matrix
    env = ShiftInvertEnv(size=A.shape[0], sigma=sigma, target_eigenvalue=target_eigenvalue, density=0.01)
    env.A = A  # Override generated matrix
    env.A_norm = np.sqrt(np.sum(A.data**2)) if issparse(A) else np.linalg.norm(A, 'fro')
    env.solver = ShiftInvertPowerRL(A, sigma)

    # Evaluate
    obs = env.reset()
    best_vec = env.state
    best_residual = np.inf

    for _ in range(1000):
        action, _ = model.predict(obs)
        obs, _, _, _ = env.step(action)
        current_residual = np.linalg.norm(A @ env.state - target_eigenvalue * env.state)

        if current_residual < best_residual:
            best_residual = current_residual
            best_vec = env.state

    # Compute cosine distance
    v1 = best_vec / (np.linalg.norm(best_vec) + 1e-8)
    v2 = actual_eigenvector / (np.linalg.norm(actual_eigenvector) + 1e-8)
    cos_sim = np.dot(v1, v2)
    cos_sim = np.clip(cos_sim, -1.0, 1.0)
    cos_dist = 2 * np.arccos(np.abs(cos_sim)) / np.pi

    print(f"Inference residual norm: {best_residual:.4e}")
    print(f"Inference eigenvector norm check: {np.linalg.norm(best_vec):.4f}")
    print(f"Inference cosine distance to actual eigenvector: {cos_dist:.6f}")

    return best_vec, best_residual, cos_dist



In [3]:

# Usage Example
# Train on diverse matrices
np.random.seed(42)
model, train_vec, train_residual, train_cos_dist = train_hybrid_eigen_solver(size=1000, sigma=0.5, density=0.01)




Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


-----------------------------
| time/              |      |
|    fps             | 278  |
|    iterations      | 1    |
|    time_elapsed    | 7    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 283        |
|    iterations           | 2          |
|    time_elapsed         | 14         |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.02695046 |
|    clip_fraction        | 0.212      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.42e+03  |
|    explained_variance   | -0.000384  |
|    learning_rate        | 0.0001     |
|    loss                 | -0.0304    |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0269    |
|    std                  | 1          |
|    value_loss           | 15.1       |
----------------------------------------
-----------

In [12]:
# Inference on a new matrix
new_A = sparse_random(1000, 1000, density=0.01, format='csr')
new_A = (new_A + new_A.T) * 0.5  # Symmetric
infer_vec, infer_residual, infer_cos_dist = infer_eigenvector(model, new_A, sigma=0.5)
print(f"Inference residual norm: {infer_residual:.4e}")
print(f"Inference eigenvector norm check: {np.linalg.norm(infer_vec):.4f}")
print(f"Inference cosine distance to actual eigenvector: {infer_cos_dist:.6f}")

Inference residual norm: 1.3801e+00
Inference eigenvector norm check: 1.0000
Inference cosine distance to actual eigenvector: 0.989847
Inference residual norm: 1.3801e+00
Inference eigenvector norm check: 1.0000
Inference cosine distance to actual eigenvector: 0.989847
